In [1]:
import torch
from torch.profiler import profile, ProfilerActivity
import pandas as pd

In [16]:
tb_file = '/users/alikaan.gueven/SDV-ML/ParticleTransformer/SDV-ML/training/tb/clip-g1-1.cbe.vbc.ac.at_26141.1754660426202800162.pt.trace.json'

In [17]:
import json

with open(tb_file, 'r') as f:
    data = json.load(f)

In [18]:
# data

In [19]:
# -------- configuration ------------------------------------------------------
trace_path = tb_file      # or leave None and use trace_dict
GPU_TAGS    = {"kernel", "memcpy", "gpukernel", "cuda"}  # tweak if needed
# -----------------------------------------------------------------------------


# --- 1. load -----------------------------------------------------------------
import json, pandas as pd, pathlib, itertools, collections, math

if trace_path is not None:
    events = json.loads(pathlib.Path(trace_path).read_text())["traceEvents"]
else:                                     # reuse the big dict you printed
    events = trace_dict["traceEvents"]

# keep only duration (“complete”) events
events = [e for e in events if e.get("ph") == "X" and "dur" in e]

# --- 2. separate CPU vs GPU ---------------------------------------------------
def is_gpu(e):
    cat  = (e.get("cat")  or "").lower()
    name = (e.get("name") or "").lower()
    return any(tag in cat or tag in name for tag in GPU_TAGS)

gpu_evts = [e for e in events if is_gpu(e)]
cpu_evts = [e for e in events if not is_gpu(e)]

# --- 3. quick diagnostics -----------------------------------------------------
if not gpu_evts:
    print("⚠️  No GPU events were recorded in this trace.")
    print("   • Did you enable ProfilerActivity.CUDA ?")
    print("   • Are you sure you loaded the *_CUDA* window, not CPU-only?")
else:
    # total active span
    t0 = min(e["ts"] for e in events)
    t1 = max(e["ts"] + e["dur"] for e in events)
    span_us = (t1 - t0) / 1_000     # μs → ms

    gpu_time = sum(e["dur"] for e in gpu_evts) / 1_000
    util_pct = 100 * gpu_time / span_us

    print(f"GPU active time : {gpu_time:,.1f} ms")
    print(f"Trace span      : {span_us:,.1f} ms")
    print(f"≈ Utilisation   : {util_pct:4.1f} %")

    # top-10 longest GPU kernels / memcpys
    top = sorted(gpu_evts, key=lambda e: -e["dur"])[:10]
    df  = pd.DataFrame({
        "cat":  [e["cat"]  for e in top],
        "name": [e["name"] for e in top],
        "ms":   [e["dur"]/1_000 for e in top],
    })
    display(df.style.format({"ms": "{:.3f}"}))


GPU active time : 2,154.7 ms
Trace span      : 2,032.0 ms
≈ Utilisation   : 106.0 %


,cat,name,ms
0,cuda_runtime,cudaStreamSynchronize,238.050
1,cuda_runtime,cudaStreamSynchronize,229.583
2,cuda_runtime,cudaStreamSynchronize,206.788
3,cuda_runtime,cudaStreamSynchronize,202.293
4,cuda_runtime,cudaLaunchKernel,23.732
5,cuda_runtime,cudaLaunchKernel,11.740
6,cuda_runtime,cudaLaunchKernel,8.155
7,kernel,"void cudnn::cnn::wgrad_alg0_engine<128, 5, 7, 3, 3, 5, false>(int, int, int, double const*, int, double*, double const*, kernel_grad_params, unsigned long long, int, double, int, int, int)",4.168
8,kernel,"void cudnn::cnn::wgrad_alg0_engine<128, 5, 7, 3, 3, 5, false>(int, int, int, double const*, int, double*, double const*, kernel_grad_params, unsigned long long, int, double, int, int, int)",4.167
9,kernel,"void cudnn::cnn::wgrad_alg0_engine<128, 5, 7, 3, 3, 5, false>(int, int, int, double const*, int, double*, double const*, kernel_grad_params, unsigned long long, int, double, int, int, int)",3.185


In [25]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [26]:
%tensorboard --logdir ../training/tb

Reusing TensorBoard on port 6007 (pid 1915), started 0:00:19 ago. (Use '!kill 1915' to kill it.)